In [19]:
import pandas as pd
import numpy as np
import os
import sqlite3
os.makedirs("raw", exist_ok=True)
os.makedirs("processed", exist_ok=True)
os.makedirs("output", exist_ok=True)
df = pd.read_csv("order_items.csv")
df.head()
print("Rows:", df.shape[0])
print("Columns:", df.shape[1])
df.info()
df.isnull().sum()
df = df.drop_duplicates()
df = df.fillna(0)
df.columns = df.columns.str.lower().str.strip()
df.columns
df['quantity'] = df['quantity'].astype(int)
df['item_price'] = df['item_price'].astype(float)
df['item_total'] = df['item_total'].astype(float)
df['calculated_total'] = df['quantity'] * df['item_price']
df['price_match_flag'] = np.where(
    df['item_total'] == df['calculated_total'], 1, 0
)
df['high_value_item'] = df['item_total'].apply(lambda x: 1 if x > 500 else 0)
df.to_csv("processed/processed_order_items.csv", index=False)
orders = df[['order_id', 'user_id']].drop_duplicates()
products = df[['product_id', 'item_price']].drop_duplicates()
order_items = df[['order_item_id', 'order_id', 'product_id',
                  'quantity', 'item_total', 'high_value_item']]
orders.to_csv("output/orders.csv", index=False)
products.to_csv("output/products.csv", index=False)
order_items.to_csv("output/order_items.csv", index=False)
conn = sqlite3.connect("output/database.sqlite")

orders.to_sql("orders", conn, if_exists="replace", index=False)
products.to_sql("products", conn, if_exists="replace", index=False)
order_items.to_sql("order_items", conn, if_exists="replace", index=False)

conn.close()
print("Orders:", orders.shape)
print("Products:", products.shape)
print("Order Items:", order_items.shape)


Rows: 43525
Columns: 7
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43525 entries, 0 to 43524
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   order_item_id  43525 non-null  object 
 1   order_id       43525 non-null  object 
 2   product_id     43525 non-null  object 
 3   user_id        43525 non-null  object 
 4   quantity       43525 non-null  int64  
 5   item_price     43525 non-null  float64
 6   item_total     43525 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 2.3+ MB
Orders: (20000, 2)
Products: (2000, 2)
Order Items: (43525, 6)
